In [35]:
import pandas as pd
import numpy as np
import pickle as pkl
from collections import Counter
import os
import re
from os import listdir
from os.path import isfile, join
import sklearn
import nltk
from nltk.corpus import stopwords 

from keras.models import Sequential
from keras import layers
import tensorflow as tf

In [2]:
INPUT_DIR = '../data/input/groundtruth/'
OUTPUT_DIR = '../data/input/groundtruth/'

CORPUS_DIR = os.path.join(OUTPUT_DIR, 'corpus')
VECTORIZED_DIR = os.path.join(OUTPUT_DIR, 'vectorized_trainset')

text_col_header = 'text'
label_col_header = 'label'

### Reading test and train data to keep them ready for testing

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# Read Train data
ip_train_file = os.path.join(INPUT_DIR, 'speechact_train.csv')
df_train = pd.read_csv(ip_train_file)
df_train = df_train.astype({text_col_header: str, label_col_header: int})

# Read Test data
ip_test_file = os.path.join(INPUT_DIR, 'speechact_test.csv')
df_test = pd.read_csv(ip_test_file)
df_test = df_test.astype({text_col_header: str, label_col_header: int})

# LSTM

Requirements
<li> Input - Vectorized Data train </li>
<li> Input - Vectorized Label train </li>

### Counter Vector with stopwords

In [4]:
# Read the vectorized train data and train label
vectorized_train_data_file = os.path.join(VECTORIZED_DIR, 'train_data_countvector_stopword.pkl')
with open(vectorized_train_data_file, 'rb') as f_ip:
    data_train = pkl.load(f_ip)
    
train_label_file = os.path.join(VECTORIZED_DIR, 'train_label_countvector_stopword.pkl')
with open(train_label_file, 'rb') as f_ip:
    label_train = pkl.load(f_ip)

#### Train and Test the model

In [20]:
# Read the vectorizer to transform the test data
vector_file = os.path.join(CORPUS_DIR, 'vector_countvector_stopword.pkl')
with open(vector_file, 'rb') as f_ip:
    vectorizer = pkl.load(f_ip)

data_test = vectorizer.transform(df_test[text_col_header])

label_test = df_test[label_col_header]
# label_test = label_test.reshape((len(label_test), 1))

In [21]:
label_train.shape

(37099, 1)

In [22]:
data_train.toarray().shape

(37099, 34154)

In [24]:
input_dim = data_train.shape[1]  # Number of features
model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 10)                341550    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 11        
Total params: 341,561
Trainable params: 341,561
Non-trainable params: 0
_________________________________________________________________


In [26]:
history = model.fit(data_train.toarray(), label_train,
                     epochs=100, verbose=True, validation_split=0.1, batch_size=10)

# score = model.evaluate(data_test.toarray(), label_test, verbose=1)


Epoch 1/100
3339/3339 [==============================] - 9s 3ms/step - loss: 1.0026e-07 - accuracy: 0.3628 - val_accuracy: 0.3558 - val_loss: 1.0273e-07
Epoch 2/100
3339/3339 [==============================] - 9s 3ms/step - loss: 1.0026e-07 - accuracy: 0.3628 - val_accuracy: 0.3558 - val_loss: 1.0273e-07
Epoch 3/100
3339/3339 [==============================] - 9s 3ms/step - loss: 1.0026e-07 - accuracy: 0.3628 - val_accuracy: 0.3558 - val_loss: 1.0273e-07
Epoch 4/100
3339/3339 [==============================] - 9s 3ms/step - loss: 1.0026e-07 - accuracy: 0.3628 - val_accuracy: 0.3558 - val_loss: 1.0273e-07
Epoch 5/100
3339/3339 [==============================] - 9s 3ms/step - loss: 1.0026e-07 - accuracy: 0.3628 - val_accuracy: 0.3558 - val_loss: 1.0273e-07
Epoch 6/100
3339/3339 [==============================] - 9s 3ms/step - loss: 1.0026e-07 - accuracy: 0.3628 - val_accuracy: 0.3558 - val_loss: 1.0273e-07
Epoch 7/100
3339/3339 [==============================] - 9s 3ms/step - loss: 1.002

3339/3339 [==============================] - 9s 3ms/step - loss: 1.0026e-07 - accuracy: 0.3628 - val_accuracy: 0.3558 - val_loss: 1.0273e-07
Epoch 55/100
3339/3339 [==============================] - 9s 3ms/step - loss: 1.0026e-07 - accuracy: 0.3628 - val_accuracy: 0.3558 - val_loss: 1.0273e-07
Epoch 56/100
3339/3339 [==============================] - 9s 3ms/step - loss: 1.0026e-07 - accuracy: 0.3628 - val_accuracy: 0.3558 - val_loss: 1.0273e-07
Epoch 57/100
3339/3339 [==============================] - 9s 3ms/step - loss: 1.0026e-07 - accuracy: 0.3628 - val_accuracy: 0.3558 - val_loss: 1.0273e-07
Epoch 58/100
3339/3339 [==============================] - 9s 3ms/step - loss: 1.0026e-07 - accuracy: 0.3628 - val_accuracy: 0.3558 - val_loss: 1.0273e-07
Epoch 59/100
3339/3339 [==============================] - 9s 3ms/step - loss: 1.0026e-07 - accuracy: 0.3628 - val_accuracy: 0.3558 - val_loss: 1.0273e-07
Epoch 60/100
3339/3339 [==============================] - 9s 3ms/step - loss: 1.0026e-07 

In [55]:
predictions_test = model.predict_classes(data_test.toarray(), verbose=1)

290/290 [==============================] - 1s 4ms/step


#### Performance metrics

In [56]:
from sklearn.metrics import f1_score

## Accuracy Measure
loss, accuracy = model.evaluate(data_train.toarray(), label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(data_test.toarray(), label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

# F1
f1_measure = f1_score(df_test[label_col_header], predictions_test, average='macro')
print('F1 macro Score: ', f1_measure)

# Classification report
from sklearn.metrics import classification_report
print(classification_report(df_test[label_col_header], predictions_test))

Training Accuracy: 0.3621
Testing Accuracy:  0.3555
F1 macro Score:  0.25816578310738997
              precision    recall  f1-score   support

           0       0.40      0.61      0.49      3911
           1       0.27      0.31      0.29      2908
           2       0.00      0.00      0.00      2456

    accuracy                           0.36      9275
   macro avg       0.22      0.31      0.26      9275
weighted avg       0.25      0.36      0.30      9275



/Users/talat/anaconda3/envs/sentgen/lib/python3.5/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Counter Vector with no stopwords

In [15]:
# Read the vectorized train data and train label
vectorized_train_data_file = os.path.join(VECTORIZED_DIR, 'train_data_countvector_nostopword.pkl')
with open(vectorized_train_data_file, 'rb') as f_ip:
    data_train = pkl.load(f_ip)
    
train_label_file = os.path.join(VECTORIZED_DIR, 'train_label_countvector_nostopword.pkl')
with open(train_label_file, 'rb') as f_ip:
    label_train = pkl.load(f_ip)

#### Train and Test the model

In [16]:
# Read the vectorizer to transform the test data
vector_file = os.path.join(CORPUS_DIR, 'vector_countvector_nostopword.pkl')
with open(vector_file, 'rb') as f_ip:
    vectorizer = pkl.load(f_ip)

data_test = vectorizer.transform(df_test[text_col_header])

In [17]:
# Run Naive Bayes
estimator = GaussianNB()
estimator.fit(data_train.toarray(), np.ravel(label_train, order='C'))
predictions_test = estimator.predict(data_test)

#### Performance metrics

In [18]:
from sklearn.metrics import f1_score

## Accuracy Measure
print('Train Accuracy', estimator.score(data_train, df_train[label_col_header]))
print('Test Accuracy', estimator.score(data_test, df_test[label_col_header]))

# F1
f1_measure = f1_score(df_test[label_col_header], predictions_test, average='macro')
print('F1 macro Score: ', f1_measure)

# Classification report
from sklearn.metrics import classification_report
print(classification_report(df_test[label_col_header], predictions_test))

Train Accuracy 0.6491280088412087
Test Accuracy 0.4821563342318059
F1 macro Score:  0.4638170698812691
              precision    recall  f1-score   support

           0       0.68      0.19      0.30      3911
           1       0.42      0.52      0.46      2908
           2       0.49      0.90      0.63      2456

    accuracy                           0.48      9275
   macro avg       0.53      0.54      0.46      9275
weighted avg       0.54      0.48      0.44      9275



### Counter Vector with minimum occurences of words (min_occurences=2)

In [19]:
# Read the vectorized train data and train label
vectorized_train_data_file = os.path.join(VECTORIZED_DIR, 'train_data_countvector_minoccurences_2.pkl')
with open(vectorized_train_data_file, 'rb') as f_ip:
    data_train = pkl.load(f_ip)
    
train_label_file = os.path.join(VECTORIZED_DIR, 'train_label_countvector_minoccurences_2.pkl')
with open(train_label_file, 'rb') as f_ip:
    label_train = pkl.load(f_ip)

#### Train and Test the model

In [20]:
# Read the vectorizer to transform the test data
vector_file = os.path.join(CORPUS_DIR, 'vector_countvector_minoccurences_2.pkl')
with open(vector_file, 'rb') as f_ip:
    vectorizer = pkl.load(f_ip)

data_test = vectorizer.transform(df_test[text_col_header])

In [21]:
# Run Naive Bayes
estimator = GaussianNB()
estimator.fit(data_train.toarray(), np.ravel(label_train, order='C'))
predictions_test = estimator.predict(data_test)

#### Performance metrics

In [22]:
from sklearn.metrics import f1_score

## Accuracy Measure
print('Train Accuracy', estimator.score(data_train, df_train[label_col_header]))
print('Test Accuracy', estimator.score(data_test, df_test[label_col_header]))

# F1
f1_measure = f1_score(df_test[label_col_header], predictions_test, average='macro')
print('F1 macro Score: ', f1_measure)

# Classification report
from sklearn.metrics import classification_report
print(classification_report(df_test[label_col_header], predictions_test))

Train Accuracy 0.685274535701771
Test Accuracy 0.554177897574124
F1 macro Score:  0.5619758483845737
              precision    recall  f1-score   support

           0       0.64      0.31      0.42      3911
           1       0.43      0.65      0.52      2908
           2       0.69      0.83      0.75      2456

    accuracy                           0.55      9275
   macro avg       0.59      0.60      0.56      9275
weighted avg       0.59      0.55      0.54      9275

